# Import libraries

In [ ]:
import multiprocessing
multiprocessing.set_start_method('spawn')

RuntimeError: context has already been set

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os.path
import sys
import numpy as np
import math
import sklearn.metrics as metrics
from tqdm import tqdm
from gym import Env, spaces
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


from tensorflow.python.keras import backend
from tensorflow.keras.backend import clear_session


import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import csv

import tensorflow as tf
import tensorflow.compat.v1 as tf1
import tensorflow.keras as K

from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import numpy as np
import time
import sys

In [ ]:
#!/usr/bin/env python
import random
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Pre-Process Data and utility funcs


In [ ]:
def read_and_decode(dataset, batch_size, is_training, data_size,n_patients):
    if is_training:
        dataset = dataset.shuffle(buffer_size=data_size, reshuffle_each_iteration=True)
        dataset = dataset.batch(batch_size, drop_remainder=False)
        dataset = dataset.repeat(None)
    else:
        #dataset = dataset.prefetch(buffer_size=data_size, reshuffle_each_iteration=True // batch_size)
        dataset = dataset.prefetch(buffer_size=data_size // batch_size)
        dataset = dataset.batch(batch_size, drop_remainder=False)
        dataset = dataset.repeat(None)
    return dataset

In [ ]:

def initialize_clinical_practice(clinical_cases_feat,clinical_cases_labels,dataset_size,n_classes,is_training,n_patients,set_distribution):

    if is_training:
        _, counts = np.unique(clinical_cases_labels, return_counts=True)

        akiec = np.squeeze(np.take(clinical_cases_feat, np.where(clinical_cases_labels == 0), axis=0))
        akiec_labels = np.squeeze(np.take(clinical_cases_labels, np.where(clinical_cases_labels == 0), axis=0))

        bcc = np.squeeze(np.take(clinical_cases_feat, np.where(clinical_cases_labels == 1), axis=0))
        bcc_labels = np.squeeze(np.take(clinical_cases_labels, np.where(clinical_cases_labels == 1), axis=0))

        bkl = np.squeeze(np.take(clinical_cases_feat, np.where(clinical_cases_labels == 2), axis=0))
        bkl_labels = np.squeeze(np.take(clinical_cases_labels, np.where(clinical_cases_labels == 2), axis=0))

        df = np.squeeze(np.take(clinical_cases_feat, np.where(clinical_cases_labels == 3), axis=0))
        df_labels = np.squeeze(np.take(clinical_cases_labels, np.where(clinical_cases_labels == 3), axis=0))

        mel = np.squeeze(np.take(clinical_cases_feat, np.where(clinical_cases_labels == 4), axis=0))
        mel_labels = np.squeeze(np.take(clinical_cases_labels, np.where(clinical_cases_labels == 4), axis=0))

        nv = np.squeeze(np.take(clinical_cases_feat, np.where(clinical_cases_labels == 5), axis=0))
        nv_labels = np.squeeze(np.take(clinical_cases_labels, np.where(clinical_cases_labels == 5), axis=0))

        vasc = np.squeeze(np.take(clinical_cases_feat, np.where(clinical_cases_labels == 6), axis=0))
        vasc_labels = np.squeeze(np.take(clinical_cases_labels, np.where(clinical_cases_labels == 6), axis=0))

        akiec_set = tf.data.Dataset.from_tensor_slices((akiec, akiec_labels)).shuffle(buffer_size=counts[0],
                                                                                      reshuffle_each_iteration=True).repeat()
        bcc_set = tf.data.Dataset.from_tensor_slices((bcc, bcc_labels)).shuffle(buffer_size=counts[1],
                                                                                reshuffle_each_iteration=True).repeat()
        bkl_set = tf.data.Dataset.from_tensor_slices((bkl, bkl_labels)).shuffle(buffer_size=counts[2],
                                                                                reshuffle_each_iteration=True).repeat()
        df_set = tf.data.Dataset.from_tensor_slices((df, df_labels)).shuffle(buffer_size=counts[3],
                                                                             reshuffle_each_iteration=True).repeat()
        mel_set = tf.data.Dataset.from_tensor_slices((mel, mel_labels)).shuffle(buffer_size=counts[4],
                                                                                reshuffle_each_iteration=True).repeat()
        nv_set = tf.data.Dataset.from_tensor_slices((nv, nv_labels)).shuffle(buffer_size=counts[5],
                                                                             reshuffle_each_iteration=True).repeat()
        vasc_set = tf.data.Dataset.from_tensor_slices((vasc, vasc_labels)).shuffle(buffer_size=counts[6],
                                                                                   reshuffle_each_iteration=True).repeat()

        dataset_train = tf.data.Dataset.sample_from_datasets([akiec_set, bcc_set, bkl_set, df_set, mel_set, nv_set, vasc_set], weights=set_distribution)
        dataset_train = dataset_train.batch(1)

    else:
        dataset_train = tf.data.Dataset.from_tensor_slices((clinical_cases_feat,clinical_cases_labels))

        dataset_train = read_and_decode(dataset_train, 1, is_training, dataset_size,n_patients)

    patients = iter(dataset_train)

    return patients


In [ ]:

def get_next_patient(patients):
    patient_scores, patient_diagnostics = patients.get_next()
    return np.squeeze(patient_scores), patient_diagnostics.numpy()[0]

In [ ]:
def torch_to_numpy(tensor):
    return tensor.data.numpy()

def numpy_to_torch(array):
    return torch.tensor(array).float()

# Shared Adam

In [ ]:
import torch as T
import torch.multiprocessing as mp
class SharedAdam(T.optim.Adam):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.99), eps=1e-8,
            weight_decay=0, state_steps = [torch.tensor([0])]):
        super(SharedAdam, self).__init__(params, lr=lr, betas=betas, eps=eps,
                weight_decay=weight_decay, state_steps = [torch.tensor([0])])

        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = 0
                state['exp_avg'] = T.zeros_like(p.data)
                state['exp_avg_sq'] = T.zeros_like(p.data)

                state['exp_avg'].share_memory_()
                state['exp_avg_sq'].share_memory_()

# A2C Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self, input_size, num_actions):
        super(Network, self).__init__()

        # Shared layers for processing combined input
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)

        # Actor head
        self.fc_actor = nn.Linear(64, num_actions)

        # Critic head
        self.fc_critic = nn.Linear(64, 1)

    def forward(self, x):
        # Shared layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # Actor head
        policy_logits = self.fc_actor(x)

        # Critic head
        value_estimate = self.fc_critic(x)

        return policy_logits, value_estimate

# A2C Environment

In [ ]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt

class DermEnv(gym.Env):
    def __init__(self, patients, vocab, n_patients):

        # Actions we can take, either skin lesion classes or don't know
        self.action_space = spaces.Discrete(len(vocab))
        self.n_classes = len(vocab)

        # Observation space - softmax + features after GAP
        self.observation_space = spaces.Box(-1*math.inf*np.ones((self.n_classes,)),math.inf*np.ones((self.n_classes,)))

        # Set shower length
        self.number_of_patients = 1
        self.n_patients = n_patients
        self.patients = patients

        # Initialize state
        n_state, n_gt = get_next_patient(self.patients)

        self.state = n_state
        self.revised_state = self.state
        self.gt = n_gt



        print("State shape:", self.state.shape)
        print("Number actions:", self.action_space) # 14 possible actions AI can play
        print("Observation space:", self.observation_space)


    def step(self, patients, action):
        ### UNKN_reward = -1
        reward_table = np.array([[2,-2,-3,	-3,	-2,	-3,	-3,	-1],
                                 [-2,3,	-4,	-4,	-2,	-4,	-4,	-1],
                                 [-2,-2, 1,	-2,	-3,	-2,	-2,	-1],
                                 [-2,-2,-2,	1,-3,	-2,	-2,	-1],
                                 [-4,-3,-5,	-5,	5,-5,	-5,	-1],
                                 [-2,-2,-2,	-2,	-3,	1,	-2,	-1],
                                 [-2,-2,-2,	-2,	-3,	-2,	1,	-1],
                                 ],np.float32)

        self.revised_state = tf.one_hot(action, self.n_classes)

        reward = reward_table[self.gt, action]

        n_state, n_gt = get_next_patient(patients)

        old_gt = self.gt

        self.state = n_state
        self.gt = n_gt

        self.number_of_patients += 1

        # Check if checking patients is done
        if self.number_of_patients >=  self.n_patients:# or old_gt != action:
            done = True
        else:
            done = False
            #reward += 1000

        return self.revised_state, self.state, reward, done, old_gt

    def reset(self, patients_val, is_training):
              # Reset clinical practice
              if is_training:
                n_state, n_gt = get_next_patient(self.patients)
              else:
                self.patients = patients_val
                n_state, n_gt = get_next_patient(self.patients)

              self.state = n_state
              self.revised_state = self.state
              self.gt = n_gt

              # Reset new practice
              self.number_of_patients = 1
              return self.state,  self.patients


# A2C Actor and Critic

In [ ]:

class ActorCriticNNAgent():
    '''
    Neural-net agent that trains using the actor-critic algorithm. The critic
    is a value function that returns expected discounted reward given the
    state as input. We use advantage defined as

        A = r + g * V(s') - V(s)

    Notation:
        A - advantage
        V - value function
        r - current reward
        g - discount factor
        s - current state
        s' - next state
    '''

    def __init__(self, vocab, lr=1e-3, df=0.5, alpha=0.5):

        # if trainable is changed to false, the model won't be updated
        self.trainable = True
        self.vocab = vocab
        self.n_classes = len(self.vocab)

        self.model = Network(519, self.n_classes)

        if isinstance(self.model, torch.nn.Module):
                    self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

        self.df = df # discount factor
        self.alpha = alpha # multiply critic updates by this factor

        # initialize replay history
        self.replay = []
    def act(self, state):
        x = state
        x = torch.tensor(x).float()
        action_values, _ = self.model.forward(x)
        action = torch.argmax(action_values).item()
        if self.trainable:
            # Convert state to NumPy array
            state_np = state.numpy() if isinstance(state, torch.Tensor) else state
            self.replay[-1]['states'].append(state_np)
            self.replay[-1]['actions'].append(action)

        return action


    def new_episode(self):

        # start a new episode in replay
        self.replay.append({'states': [], 'actions': [], 'rewards': []})

    def store_reward(self, r):
        # insert 0s for actions that received no reward; end with reward r
        episode = self.replay[-1]
        episode['rewards'] +=  [r]

    def _calculate_discounted_rewards(self):
        # calculate and store discounted rewards per episode
        for episode in self.replay:

            R = episode['rewards']
            R_disc = []
            R_sum = 0
            for r in R[::-1]:
                R_sum = r + self.df * R_sum
                R_disc.insert(0, R_sum)

            episode['rewards_disc'] = R_disc

    def update(self):

        assert(self.trainable)

        episode_losses = torch.tensor(0.0)
        N = len(self.replay)
        self._calculate_discounted_rewards()

        for episode in self.replay:

            O = episode['states']

            A = episode['actions']
            R = torch.tensor(episode['rewards']).float()
            R_disc = torch.tensor(episode['rewards_disc']).float()
            T = len(R_disc)
            O_array = np.array(O)
            x = torch.tensor(O_array).float()
            Y1, Y2 = self.model.forward(x)
            pi = Y1
            Vs_curr = Y2.view(-1)

            # log probabilities of selected actions
            log_prob = torch.log(pi[range(T), A])

            # advantage of selected actions over expected reward given state
            Vs_next = torch.cat((Vs_curr[1:], torch.tensor([0.])))
            adv = R + self.df * Vs_next - Vs_curr

            # ignore gradients so the critic isn't affected by actor loss
            adv = adv.detach()

            # actor loss is -1 * advantage-weighted sum of log likelihood
            # critic loss is the SE between values and discounted rewards
            actor_loss = -torch.dot(log_prob, adv)
            critic_loss = torch.sum((R_disc - Vs_curr) ** 2)
            episode_losses += actor_loss + critic_loss * self.alpha

        # backward pass
        self.optimizer.zero_grad()
        loss = episode_losses / N
        loss.backward()
        self.optimizer.step()

        # reset the replay history
        self.replay = []

    def train_parallel(self, num_actors, iterations, episodes, patients):
        self.patients = patients
        self.envs = [DermEnv(patients=self.patients, vocab=self.vocab, n_patients=150) for _ in range(num_actors)]
        self.actors = [ParallelActor(agent = self, env = self.envs[i], index = i, iterations = iterations, episodes = episodes, patients = self.patients) for i in range(num_actors)]

        for actor in self.actors:
            actor.start()
        for actor in self.actors:
            actor.join()


    def test(self, iterations, patients):

        self.trainable = False
        self.patients = patients

        error = np.array([])
        true_label = np.array([])

        rewards = []
        rewards_history = []
        self.episode_reward_history = []
        self.episode_val_reward_history = []
        self.validation_bacc_history = []
        self.mel_history = []
        self.unk_history = []

        mel_count = 0
        unk_count = 0
        best_bacc = 0
        best_reward = -1 * math.inf

        for _ in range(iterations):

            total_reward = 0
            done = False
            state, patients_val = self.env.reset(patients_val = self.patients, is_training =  False)

            while not done:
                true_label = np.append(true_label, self.env.gt)
                action = self.act(state)
                error = np.append(error, action)

                if vocab[action] == 'mel':
                    mel_count += 1
                elif vocab[action] == 'unkn':
                    unk_count += 1

                _, state, reward, done, _ = self.env.step(patients_val, action)
                total_reward += reward

            rewards.append(total_reward)

            print('The reward of the validation episode was ', total_reward)
            print('The balanced accuracy was ', metrics.balanced_accuracy_score(true_label, error))
            print('The accuracy was ', metrics.accuracy_score(true_label, error))

            self.episode_val_reward_history.append(total_reward)
            self.validation_bacc_history.append(metrics.balanced_accuracy_score(true_label, error))
            self.mel_history.append(mel_count)
            self.unk_history.append(unk_count)

            if best_bacc < metrics.balanced_accuracy_score(true_label, error):
                history_report_bacc = metrics.classification_report(true_label,error,digits=3)
                history_cov_bacc = metrics.confusion_matrix(true_label,error)
                best_bacc = metrics.balanced_accuracy_score(true_label, error)

            if best_reward < total_reward:
                history_cov_reward = metrics.confusion_matrix(true_label, error)
                history_report_reward = metrics.classification_report(true_label,error,digits=3)
                best_reward = total_reward

        print('The scores for best validation BAcc are:')
        print(history_report_bacc)
        print(history_cov_bacc)
        print('The best BAcc was ',best_bacc)

        print('The scores for best validation Reward are:')
        print(history_report_reward)
        print(history_cov_reward)
        print('The best reward was ', best_reward)

    def plot_results(self):
        if self.trainable:
            plt.figure(1)
            plt.plot(self.results)
            plt.xlabel('Episode')
            plt.ylabel('Reward')
            plt.title('Training Progress')
            plt.show()
        else:
            plt.figure(2)
            plt.plot(self.episode_val_reward_history)
            plt.xlabel('Episodes')
            plt.ylabel('Reward Per Episode - Val')
            plt.show()

            plt.figure(3)
            plt.plot(self.validation_bacc_history)
            plt.xlabel('Episodes')
            plt.ylabel('RL BAcc')
            plt.show()

            plt.figure(4)
            plt.plot(self.mel_history)
            plt.xlabel('Episodes')
            plt.ylabel('Number Melanoma Decisions')
            plt.show()

# Importing Dataset

In [ ]:
#### Import Datasets ####
#tf1.enable_eager_execution()

database = pd.read_csv('vectorDBNoSpace.csv')

print(database)

database.head()

labels = np.asarray(database['dx'])

print(labels)

labels[labels == 'scc'] = 'akiec'

le = preprocessing.LabelEncoder()

le.fit(labels)

vocab = le.classes_

n_words = len(vocab)

features1 = np.load("nmed_rn34_ham10k_vectors.npy")

features2 = pd.read_csv("vectorDBNoSpace.csv")

features2.pop('dx')

features2 = np.asarray(features2, dtype='float32')

features = np.concatenate([features1,features2],axis=1)


_, counts = np.unique(labels, return_counts=True)

print(counts)

counts = counts/np.sum(counts)


labels_cat = le.transform(labels)

print(labels_cat.dtype)

train_feat, val_feat, train_labels, val_labels = train_test_split(features, labels_cat, test_size=0.2,
                                                                      random_state=111,stratify = labels_cat)


_, count_train = np.unique(val_labels, return_counts=True)

print(count_train)

patients = initialize_clinical_practice(train_feat,train_labels,train_labels.shape[0],True,n_words, 150,counts)

patients_val = initialize_clinical_practice(val_feat,val_labels,val_labels.shape[0],False,n_words, 150,counts)



              akiec       bcc       bkl            df       mel        nv  \
0      1.239820e-04  0.001757  0.071755  2.613447e-02  0.004628  0.895575   
1      1.564469e-03  0.000353  0.582653  1.401107e-02  0.000817  0.400582   
2      7.310000e-07  0.000002  0.005028  5.290000e-07  0.002840  0.992129   
3      3.012000e-03  0.027224  0.293466  3.917728e-02  0.062816  0.561710   
4      1.044428e-03  0.007633  0.776036  3.400000e-05  0.197736  0.017514   
...             ...       ...       ...           ...       ...       ...   
10010  2.074046e-03  0.001095  0.002900  4.630000e-05  0.991719  0.000645   
10011  9.210000e-06  0.000346  0.001581  4.070000e-05  0.308520  0.689383   
10012  5.650000e-05  0.001307  0.977360  3.800000e-05  0.000746  0.020455   
10013  3.850000e-06  0.000040  0.004911  2.747410e-04  0.214675  0.780093   
10014  2.450000e-05  0.000041  0.013609  5.980000e-05  0.091868  0.894389   

               vasc   dx  
0      2.630000e-05   nv  
1      1.920000e-05  

In [ ]:
class ParallelActor(mp.Process):
    def __init__(self, env, agent, index, iterations, episodes, patients):
        super(ParallelActor, self).__init__()
        self.env = env
        self.agent = agent
        self.index = index
        self.iterations = iterations
        self.episodes = episodes
        self.patients = patients

    def run(self):
        self.agent.env = self.env  # Assign the environment to the agent
        for iter in range(self.iterations):
            rewards = []
            for ep in range(self.episodes):
                state, patients = self.agent.env.reset(patients_val=[], is_training=True)
                self.agent.new_episode()
                total_reward = 0
                done = False
                while not done:
                    action = self.agent.act(state)
                    _, state, reward, done, _ = self.agent.env.step(patients, action)
                    self.agent.store_reward(reward)
                    total_reward += reward
                rewards.append(total_reward)
                self.agent.update()
            print(f"Actor {self.index}: Mean total reward / episode: {np.mean(rewards)}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# ||

# Start A2C

In [ ]:

agent = ActorCriticNNAgent(vocab = vocab)
agent.train_parallel(5, 1,  1, patients = patients)
agent.plot_results()
#avg_reward = agent.test(iterations = 150, patients = patients_val)
#agent.plot_results()

State shape: (519,)
Number actions: Discrete(7)
Observation space: Box(-inf, inf, (7,), float32)
State shape: (519,)
Number actions: Discrete(7)
Observation space: Box(-inf, inf, (7,), float32)
State shape: (519,)
Number actions: Discrete(7)
Observation space: Box(-inf, inf, (7,), float32)
State shape: (519,)
Number actions: Discrete(7)
Observation space: Box(-inf, inf, (7,), float32)
State shape: (519,)
Number actions: Discrete(7)
Observation space: Box(-inf, inf, (7,), float32)


InvalidArgumentError: Cannot convert a Tensor of dtype variant to a NumPy array.